In [ ]:
import sirf.STIR as pet

from sirf.Utilities import examples_data_path
import os
import sys
from cil.optimisation.functions import KullbackLeibler, OperatorCompositionFunction, IndicatorBox
cil_path = '/home/user/StochasticDev/Hackathon-000-Stochastic-Algorithms/cil/'
sys.path.append(cil_path)
from NewFISTA import ISTA
from NewSubsetSumFunction import SAGFunction, SAGAFunction, SVRGFunction, LSVRGFunction
import numpy as np

pet.set_verbosity(0)
pet.AcquisitionData.set_storage_scheme("memory")
msg = pet.MessageRedirector(info=None, warn=None, errr=None)

data_path = os.path.join(examples_data_path('PET'), 'thorax_single_slice')
image = pet.ImageData(os.path.join(data_path, 'emission.hv'))
attn_image = pet.ImageData(os.path.join(data_path, 'attenuation.hv'))
template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))
acq_model_for_attn = pet.AcquisitionModelUsingRayTracingMatrix()
asm_attn = pet.AcquisitionSensitivityModel(attn_image, acq_model_for_attn)
asm_attn.set_up(template)
attn_factors = asm_attn.forward(template.get_uniform_copy(1))
asm_attn = pet.AcquisitionSensitivityModel(attn_factors)
acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
acq_model.set_num_tangential_LORs(10)
acq_model.set_acquisition_sensitivity(asm_attn)
acq_model.set_up(template,image)
acquired_data = acq_model.forward(image)
acquired_data.fill(np.random.poisson(acquired_data.as_array()))
background_term = acquired_data.get_uniform_copy(acquired_data.max()/10)

initial = image.get_uniform_copy(0)

data_fit = KullbackLeibler(b=acquired_data, eta = background_term)

f = OperatorCompositionFunction(data_fit,acq_model)

step_size = 0.1

num_epochs = 10

G = IndicatorBox(lower=0)

ista = ISTA(initial=initial, f=f, g=G,
                            step_size=step_size, update_objective_interval=1,
                            max_iteration=1e10, check_convergence_criterion=False)
ista.run(num_epochs)
from cil.utilities.display import show2D
show2D(ista.solution)

In [ ]:
from matplotlib import pyplot as plt

n_subsets = 8

dim = acquired_data.dimensions()

print('data dimensions: %d x %d x %d x %d' % dim) 

f = []

for i in range(n_subsets):
    
    views = np.arange(i, dim[2], n_subsets)
    
    data_subset = acquired_data.get_subset(views)
    #print(views,'\n', data_subset.shape)
    eta_sirf = background_term.get_subset(views) # background is constant atm
    
    # confirm next lines with Kris/Evgueni
    acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
    acq_model.set_num_tangential_LORs(10)
    asm_attn = pet.AcquisitionSensitivityModel(attn_factors.get_subset(views)) 
    acq_model.set_acquisition_sensitivity(asm_attn)
    acq_model.set_up(data_subset,image)  
                
    A_i = acq_model
    data_fit_i = KullbackLeibler(b = data_subset, eta = eta_sirf) 
    f.append(OperatorCompositionFunction(data_fit_i,A_i))

In [ ]:
from NewSubsetSumFunction import SAGFunction, SVRGFunction, LSVRGFunction

F_sag = SAGFunction(f)
initial = image.allocate(1)
F_sag.memory_reset()
sag = ISTA(initial=initial, 
            f=F_sag,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000, check_convergence_criterion=False)
sag.run(num_epochs * n_subsets, verbose=1)

F_saga = SAGAFunction(f)
initial = image.allocate(1.)
F_saga.memory_reset()
saga = ISTA(initial=initial, f=F_saga, g=G,
                step_size=step_size, update_objective_interval=n_subsets,
                max_iteration=1e10, check_convergence_criterion=False)
saga.run(num_epochs * n_subsets, verbose=1)

F_svrg = SVRGFunction(f)
initial = image.allocate(1)
F_svrg.memory_reset()
svrg = ISTA(initial=initial, 
            f=F_svrg,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000, check_convergence_criterion=False)
svrg.run(num_epochs * n_subsets, verbose=1)
F_lsvrg = LSVRGFunction(f)
initial = image.allocate(1)
F_lsvrg.memory_reset()
lsvrg = ISTA(initial=initial, 
            f=F_lsvrg,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000, check_convergence_criterion=False)
lsvrg.run(num_epochs * n_subsets, verbose=1)

In [ ]:
# With diagonal preconditioning

# D(x) = diag(x / A^T \1)
acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
acq_model.set_num_tangential_LORs(10)
acq_model.set_matrix(acq_model.get_matrix().set_restrict_to_cylindrical_FOV(False))
acq_model.set_up(template,image)
cst = acq_model.adjoint(acquired_data.get_uniform_copy(1))
precond = lambda i, x: (x+1e-8).divide(cst)

F_sag = SAGFunction(f, precond=precond)
initial = image.allocate(1)
F_sag.memory_reset()
sag_precond = ISTA(initial=initial, 
            f=F_sag,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000, check_convergence_criterion=False)
sag_precond.run(num_epochs * n_subsets, verbose=1)

F_saga = SAGAFunction(f, precond=precond)
initial = image.allocate(1.)
F_saga.memory_reset()
saga_precond = ISTA(initial=initial, f=F_saga, g=G,
                step_size=step_size, update_objective_interval=n_subsets,
                max_iteration=1e10, check_convergence_criterion=False)
saga_precond.run(num_epochs * n_subsets, verbose=1)

F_svrg = SVRGFunction(f, precond=precond)
initial = image.allocate(1)
F_svrg.memory_reset()
svrg_precond = ISTA(initial=initial, 
            f=F_svrg,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000, check_convergence_criterion=False)
svrg_precond.run(num_epochs * n_subsets, verbose=1)
F_lsvrg = LSVRGFunction(f, precond=precond)
initial = image.allocate(1)
F_lsvrg.memory_reset()
lsvrg_precond = ISTA(initial=initial, 
            f=F_lsvrg,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000, check_convergence_criterion=False)
lsvrg_precond.run(num_epochs * n_subsets, verbose=1)

In [ ]:
show2D([sag.solution,
       saga.solution,
       svrg.solution, 
       lsvrg.solution,
       sag_precond.solution,
       saga_precond.solution,
       svrg_precond.solution, 
       lsvrg_precond.solution], 
       origin="upper", 
       title=["SAG", "SAGA",  "SVRG", "L-SVRG","SAG precond", "SAGA precond",  "SVRG precond", "L-SVRG precond"],
       num_cols=4,
       cmap='bone')

plt.figure(figsize=(12,8))
plt.plot(sag.loss,label='sag',linestyle = '-')
plt.plot(sag_precond.loss,label='sag_precond',linestyle = '--')
plt.plot(saga.loss,label='saga',linestyle = '-')
plt.plot(saga_precond.loss,label='saga_precond',linestyle = '--')
plt.plot(svrg.loss,label='svrg',linestyle = '-')
plt.plot(svrg_precond.loss,label='svrg_precond',linestyle = '--')
plt.plot(lsvrg.loss,label='lsvrg',linestyle = '-')
plt.plot(lsvrg_precond.loss,label='lsvrg_precond',linestyle = '--')
plt.semilogy()
plt.legend(loc=3)
plt.show()

In [ ]:
print(sag_precond.loss)
print(saga_precond.loss)
print(svrg_precond.loss)
print(lsvrg_precond.loss)